In [1]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch import nn

import dataset
import rendering
import model
import train

In [2]:
test_origin, test_direction, test_target_px_values = dataset.get_rays('../dataset/fox', mode='test')

Mode: test
Image files length: 10
Pose files length: 10
Intrinsic files length: 10
Image size: (400, 400, 4)
(10, 400, 400, 4)


In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
nerf_model = torch.load('models/model_nerf').to(device)
nerf_model.eval()

Nerf(
  (block1): Sequential(
    (0): Linear(in_features=63, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=256, bias=True)
    (9): ReLU()
  )
  (block2): Sequential(
    (0): Linear(in_features=319, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=257, bias=True)
  )
  (rgb_head): Sequential(
    (0): Linear(in_features=283, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=3, bias=True)
    (3): Si

### running the model for inference takes 8GB ram
- images are 400 * 400 = 160000 rays for each pixel
- for each ray we need 100 binned samples
- 400 * 400 * 100 = 16 million points points into neural network


#### instead of doing 16 million at the same time we will use batching to address the memory issue
- therefore instead of 16M we do 1.6M in each batch 
- this one the main issues with nerf: this causes the FPS of nerf to be very low
    - about 0.5 to 0.1 fps for any practical application
    - movies are roughly 24 fps, therefore for real time we need to decrease inference times
    - recently new methods have made it possible to render at 200 FPS!

In [9]:
def test(model, origin, direction, tn, tf, nb_bins=100, chunk_size=10):
   
    with torch.inference_mode():
        model.eval()
        origin = origin.chunk(chunk_size)
        direction = direction.chunk(chunk_size)
        
        for origin_batch, direction_batch in zip(origin, direction): #zip to interate both lists at the same time
            rendering.rendering(model, origin_batch, direction_batch, tn, tf, nb_bins, device=origin_batch.device)
        

### Chunking the input tensors into 1/10 their size

In [10]:
print(f"Pre-Chunking shape:{test_origin[0].shape}")
print(f"After Chunking size:{torch.from_numpy(test_origin[0]).chunk(10)[0].shape}")
#almost 0.1 times

Pre-Chunking shape:(160000, 3)
After Chunking size:torch.Size([16000, 3])


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#hyperparams
tn = 8
tf = 12
nb_epochs = 15
learning_rate = 1e-3
gamma = 0.5
nb_bins = 100

In [12]:
test(nerf_model,
     torch.from_numpy(test_origin[0]).to(device).float(),
     torch.from_numpy(test_direction[0]).to(device).float(),
     tn, tf,
     nb_bins=100,
     chunk_size=10)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.53 GiB. GPU 0 has a total capacity of 5.79 GiB of which 129.06 MiB is free. Including non-PyTorch memory, this process has 5.66 GiB memory in use. Of the allocated memory 5.55 GiB is allocated by PyTorch, and 18.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)